# State-Space Control and LSTM-Based Controller Design
## Vertical Acceleration Control for Autonomous Systems

This notebook implements and compares two control approaches:
1. **Linear Quadratic Regulator (LQR)** - Classical optimal control with state-space models
2. **LSTM-Based Controller** - Data-driven neural network controller

The goal is to control vertical acceleration (and thus position/velocity) of a system with normalized command inputs.

In [ ]:
# ============================================================================
# IMPORTS AND SETUP
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)

# System parameters
dt = 0.05  # Sampling time (seconds)
g = 9.81   # Gravitational acceleration (m/s^2)

print(f"System configured with dt={dt}s, g={g}m/s²")

## Section 1: Data Loading and Exploration

In [ ]:
# Load experimental data
df_in = pd.read_csv('bdd_in_mat_05.csv')
df_out = pd.read_csv('bdd_out_mat_05.csv')

print(f"Input data shape: {df_in.shape}")
print(f"Output data shape: {df_out.shape}")
print(f"\nFirst 5 input values (first sample): {df_in.iloc[0, :5].values}")
print(f"First 5 output values (first sample): {df_out.iloc[0, :5].values}")

In [ ]:
# Plot raw data exploration
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# All input signals
axes[0, 0].plot(df_in.T, alpha=0.3, color='blue')
axes[0, 0].set_title('All Input Signals (Normalized Commands)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Time Step')
axes[0, 0].set_ylabel('Normalized Command')
axes[0, 0].grid(True, alpha=0.3)

# All output signals
axes[0, 1].plot(df_out.T, alpha=0.3, color='green')
axes[0, 1].set_title('All Output Signals (Acceleration)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Time Step')
axes[0, 1].set_ylabel('Acceleration (m/s²)')
axes[0, 1].grid(True, alpha=0.3)

# First sample input vs output
time_steps = np.arange(len(df_in.iloc[0]))
ax1 = axes[1, 0]
ax2 = ax1.twinx()
ax1.plot(df_in.iloc[0].values, label='Input Command', color='blue', linewidth=2)
ax2.plot(df_out.iloc[0].values, label='Output Acceleration', color='red', linewidth=2)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Input (Normalized)', color='blue')
ax2.set_ylabel('Output (m/s²)', color='red')
axes[1, 0].set_title('First Sample: Input vs Output', fontsize=12, fontweight='bold')
ax1.tick_params(axis='y', labelcolor='blue')
ax2.tick_params(axis='y', labelcolor='red')
ax1.grid(True, alpha=0.3)

# Scatter plot for correlation
axes[1, 1].scatter(df_in.iloc[0].values, df_out.iloc[0].values, alpha=0.5, s=10)
axes[1, 1].set_xlabel('Input Command (Normalized)')
axes[1, 1].set_ylabel('Output Acceleration (m/s²)')
axes[1, 1].set_title('Input-Output Correlation', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Data exploration complete")

## Section 2: Data Preprocessing and Normalization

In [ ]:
# Outlier detection and removal using Z-score
df_in_cleaned = df_in.copy()
df_out_cleaned = df_out.copy()

z_score_threshold = 3
z_scores_in = zscore(df_in_cleaned, axis=1, nan_policy='omit')
z_scores_out = zscore(df_out_cleaned, axis=1, nan_policy='omit')

outlier_rows = (np.abs(z_scores_in) > z_score_threshold).any(axis=1) | \
               (np.abs(z_scores_out) > z_score_threshold).any(axis=1)

rows_to_keep = ~outlier_rows
df_in_cleaned = df_in_cleaned[rows_to_keep].reset_index(drop=True)
df_out_cleaned = df_out_cleaned[rows_to_keep].reset_index(drop=True)

print(f"Original samples: {len(df_in)}")
print(f"Outliers removed: {np.sum(outlier_rows)}")
print(f"Remaining samples: {len(df_in_cleaned)}")

# Normalize input data (MinMax scaling to [0,1])
X = df_in_cleaned.to_numpy()
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

# Normalize output data
# Shift by initial value, then normalize by max absolute value
y = df_out_cleaned.to_numpy()
y_shifted = y - y[:, 0:1]
global_max_abs_y = np.max(np.abs(y_shifted))
y_normalized = y_shifted / global_max_abs_y

print(f"\nInput shape: {X_scaled.shape}")
print(f"Output shape: {y_normalized.shape}")
print(f"Global max acceleration: {global_max_abs_y:.4f} m/s²")

In [ ]:
# Reshape for LSTM (samples, timesteps, features)
n_samples, n_timesteps = X_scaled.shape
X_reshaped = X_scaled.reshape(n_samples, n_timesteps, 1)
y_reshaped = y_normalized.reshape(n_samples, n_timesteps, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_reshaped, test_size=0.2, random_state=42
)

print(f"Training set: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Test set: X_test {X_test.shape}, y_test {y_test.shape}")

## Section 3: Forward LSTM Model (System Dynamics)

This model learns the system dynamics: acceleration = f(command history)

In [ ]:
# Build forward LSTM model (predicts acceleration from commands)
forward_model = Sequential([
    LSTM(32, input_shape=(n_timesteps, 1), return_sequences=True, activation='relu'),
    LSTM(16, return_sequences=True, activation='relu'),
    Dense(1, activation='linear')  # Linear output for acceleration
])

forward_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
print("Forward Model Architecture:")
forward_model.summary()

# Train forward model
print("\n" + "="*60)
print("Training Forward LSTM Model...")
print("="*60)
history_forward = forward_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=25,
    batch_size=16,
    verbose=1
)

# Evaluate
test_loss = forward_model.evaluate(X_test, y_test, verbose=0)
print(f"\nForward Model Test Loss: {test_loss:.6f}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

axes[0].plot(history_forward.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history_forward.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Forward Model: Training History', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

axes[1].plot(history_forward.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history_forward.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Forward Model: Mean Absolute Error', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Forward model training complete")

## Section 4: Forward Model Validation and Prediction Analysis

In [ ]:
# Get predictions on test set
y_pred_normalized = forward_model.predict(X_test, verbose=0)

# Denormalize predictions
y_pred_real = y_pred_normalized * global_max_abs_y
y_test_real = y_test * global_max_abs_y

# Denormalize by adding back gravity baseline
y_pred_accel = y_pred_real
y_test_accel = y_test_real

# Calculate error metrics
mse = mean_squared_error(y_test_accel.flatten(), y_pred_accel.flatten())
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_test_accel.flatten() - y_pred_accel.flatten()))

print(f"Forward Model Performance Metrics:")
print(f"  RMSE: {rmse:.6f} m/s²")
print(f"  MAE:  {mae:.6f} m/s²")
print(f"  MSE:  {mse:.6f}")

# Select a test sample for detailed analysis
sample_idx = 2
y_test_sample = y_test_accel[sample_idx].squeeze()
y_pred_sample = y_pred_accel[sample_idx].squeeze()
x_test_sample = X_test[sample_idx].squeeze()

# Calculate velocity and position by integration
def integrate_acceleration(accel, dt=0.05):
    """Double integrate acceleration to get velocity and position"""
    velocity = np.cumsum(accel) * dt
    position = np.cumsum(velocity) * dt
    return position, velocity

pos_real, vel_real = integrate_acceleration(y_test_sample, dt)
pos_pred, vel_pred = integrate_acceleration(y_pred_sample, dt)

# Create comprehensive comparison plot
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(4, 2, hspace=0.3, wspace=0.3)

# Row 1: Command vs Acceleration
ax1 = fig.add_subplot(gs[0, :])
ax1_twin = ax1.twinx()
ax1.plot(x_test_sample[:200], label='Command Input', color='blue', linewidth=2, alpha=0.7)
ax1_twin.plot(y_test_sample[:200], label='Real Acceleration', color='red', linewidth=2, alpha=0.7)
ax1_twin.plot(y_pred_sample[:200], label='Predicted Acceleration', color='red', linewidth=2, 
              alpha=0.4, linestyle='--')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Command (Normalized)', color='blue')
ax1_twin.set_ylabel('Acceleration (m/s²)', color='red')
ax1.set_title('Test Sample: Command vs Acceleration (Forward Model)', fontweight='bold', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='y', labelcolor='blue')
ax1_twin.tick_params(axis='y', labelcolor='red')
fig.legend(loc='upper right', bbox_to_anchor=(0.98, 0.96), fontsize=10)

# Row 2: Acceleration comparison
ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(y_test_sample, label='Real', linewidth=2, color='green')
ax2.plot(y_pred_sample, label='Predicted', linewidth=2, color='orange', alpha=0.7, linestyle='--')
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Acceleration (m/s²)')
ax2.set_title('Acceleration: Real vs Predicted', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Acceleration error
ax3 = fig.add_subplot(gs[1, 1])
accel_error = np.abs(y_test_sample - y_pred_sample)
ax3.plot(accel_error, color='red', linewidth=2, alpha=0.7)
ax3.fill_between(range(len(accel_error)), accel_error, alpha=0.3, color='red')
ax3.set_xlabel('Time Step')
ax3.set_ylabel('Absolute Error (m/s²)')
ax3.set_title(f'Acceleration Error (Mean: {mae:.4f} m/s²)', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Row 3: Velocity
ax4 = fig.add_subplot(gs[2, 0])
ax4.plot(vel_real, label='Real', linewidth=2, color='green')
ax4.plot(vel_pred, label='Predicted', linewidth=2, color='orange', alpha=0.7, linestyle='--')
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Velocity (m/s)')
ax4.set_title('Integrated Velocity', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Velocity error
ax5 = fig.add_subplot(gs[2, 1])
vel_error = np.abs(vel_real - vel_pred)
ax5.plot(vel_error, color='purple', linewidth=2, alpha=0.7)
ax5.fill_between(range(len(vel_error)), vel_error, alpha=0.3, color='purple')
ax5.set_xlabel('Time Step')
ax5.set_ylabel('Velocity Error (m/s)')
ax5.set_title(f'Velocity Error (Max: {np.max(vel_error):.4f} m/s)', fontweight='bold')
ax5.grid(True, alpha=0.3)

# Row 4: Position
ax6 = fig.add_subplot(gs[3, 0])
ax6.plot(pos_real, label='Real', linewidth=2, color='green')
ax6.plot(pos_pred, label='Predicted', linewidth=2, color='orange', alpha=0.7, linestyle='--')
ax6.set_xlabel('Time Step')
ax6.set_ylabel('Position (m)')
ax6.set_title('Integrated Position', fontweight='bold')
ax6.legend()
ax6.grid(True, alpha=0.3)

# Position error
ax7 = fig.add_subplot(gs[3, 1])
pos_error = np.abs(pos_real - pos_pred)
ax7.plot(pos_error, color='brown', linewidth=2, alpha=0.7)
ax7.fill_between(range(len(pos_error)), pos_error, alpha=0.3, color='brown')
ax7.set_xlabel('Time Step')
ax7.set_ylabel('Position Error (m)')
ax7.set_title(f'Position Error (Max: {np.max(pos_error):.4f} m)', fontweight='bold')
ax7.grid(True, alpha=0.3)

plt.show()

print(f"\n✓ Forward model analysis complete (sample {sample_idx})")

## Section 5: State-Space Model and LQR Controller Design

**Theory:** We model the system as a discrete-time linear system:
- **State:** x = [position, velocity]ᵀ
- **Dynamics:** x(k+1) = A·x(k) + B·u(k)
- **Controller:** u(k) = -K·(x(k) - x_target)

The optimal feedback gain K is computed by solving the Discrete Algebraic Riccati Equation (DARE).

In [ ]:
# Define discrete-time state-space matrices
# State: [position, velocity]
# Dynamics:
#   position(k+1) = position(k) + velocity(k)*dt + 0.5*acceleration(k)*dt²
#   velocity(k+1) = velocity(k) + acceleration(k)*dt

A_lqr = np.array([[1, dt],
                  [0, 1]], dtype=float)

B_lqr = np.array([[0.5 * dt**2],
                  [dt]], dtype=float)

print("Linear System Matrices:")
print(f"A (State Transition):\n{A_lqr}\n")
print(f"B (Input Matrix):\n{B_lqr}\n")

# Define cost matrices for LQR
# Q: penalizes state deviations
# R: penalizes control effort
Q_lqr = np.array([[100, 0],      # Penalize position error heavily
                  [0, 10]], dtype=float)   # Moderate velocity penalty
R_lqr = np.array([[1]], dtype=float)       # Small control penalty

print("Cost Matrices:")
print(f"Q (State Cost):\n{Q_lqr}\n")
print(f"R (Control Cost):\n{R_lqr}\n")

# Solve Discrete Algebraic Riccati Equation (DARE)
P_lqr = scipy.linalg.solve_discrete_are(A_lqr, B_lqr, Q_lqr, R_lqr)

# Compute optimal feedback gain K
K_lqr = np.linalg.inv(R_lqr + B_lqr.T @ P_lqr @ B_lqr) @ (B_lqr.T @ P_lqr @ A_lqr)

print("DARE Solution:")
print(f"P (Cost-to-go matrix):\n{P_lqr}\n")
print(f"K (Optimal Feedback Gain):\n{K_lqr}\n")

# Display controller interpretation
print("Control Law Interpretation:")
print(f"u = -K·(x - x_target)")
print(f"  = -[{K_lqr[0,0]:.4f}, {K_lqr[0,1]:.4f}] · ([position; velocity] - target)")
print(f"\nThis means:")
print(f"  - Negative position error → {-K_lqr[0,0]:.4f}× feedback (proportional to position)")
print(f"  - Negative velocity error → {-K_lqr[0,1]:.4f}× feedback (damping)")

print("\n✓ LQR controller designed")

## Section 6: LQR Controller Simulation

In [ ]:
# Simulate LQR controller to reach target altitude
target_height = 10.0
sim_duration = 20.0
num_steps_sim = int(sim_duration / dt)

# Initialize state
state_lqr = np.array([[0.0], [0.0]])  # [position, velocity]

# Storage for results
time_lqr = np.linspace(0, sim_duration, num_steps_sim)
pos_lqr = np.zeros(num_steps_sim)
vel_lqr = np.zeros(num_steps_sim)
acc_lqr = np.zeros(num_steps_sim)
control_lqr = np.zeros(num_steps_sim)

# Target state
target_state = np.array([[target_height], [0.0]])

# Simulation loop
for i in range(num_steps_sim):
    # State error
    error_state = state_lqr - target_state
    
    # Compute control input
    u_lqr = -K_lqr @ error_state
    control_lqr[i] = u_lqr[0, 0]
    
    # Store state
    pos_lqr[i] = state_lqr[0, 0]
    vel_lqr[i] = state_lqr[1, 0]
    acc_lqr[i] = u_lqr[0, 0]
    
    # Update state using system dynamics
    state_lqr = A_lqr @ state_lqr + B_lqr @ u_lqr

print(f"LQR Simulation Results:")
print(f"  Final position: {pos_lqr[-1]:.4f} m (target: {target_height} m)")
print(f"  Final velocity: {vel_lqr[-1]:.4f} m/s")
print(f"  Max acceleration: {np.max(np.abs(acc_lqr)):.4f} m/s²")
print(f"  Max control effort: {np.max(np.abs(control_lqr)):.4f}")

# Plot LQR simulation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Position
axes[0, 0].plot(time_lqr, pos_lqr, linewidth=2.5, color='green', label='Position')
axes[0, 0].axhline(y=target_height, color='red', linestyle='--', linewidth=2, label='Target')
axes[0, 0].fill_between(time_lqr, pos_lqr, target_height, alpha=0.2, color='green')
axes[0, 0].set_ylabel('Position (m)')
axes[0, 0].set_title('LQR: Position Trajectory', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Velocity
axes[0, 1].plot(time_lqr, vel_lqr, linewidth=2.5, color='blue')
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0, 1].fill_between(time_lqr, vel_lqr, alpha=0.2, color='blue')
axes[0, 1].set_ylabel('Velocity (m/s)')
axes[0, 1].set_title('LQR: Velocity Trajectory', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Acceleration (Control input)
axes[1, 0].plot(time_lqr, acc_lqr, linewidth=2.5, color='purple')
axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[1, 0].fill_between(time_lqr, acc_lqr, alpha=0.2, color='purple')
axes[1, 0].set_ylabel('Acceleration (m/s²)')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_title('LQR: Control Input (Acceleration)', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Phase portrait (velocity vs position)
axes[1, 1].plot(pos_lqr, vel_lqr, linewidth=2.5, color='cyan')
axes[1, 1].scatter([pos_lqr[0]], [vel_lqr[0]], s=100, color='green', marker='o', 
                   label='Start', zorder=5)
axes[1, 1].scatter([pos_lqr[-1]], [vel_lqr[-1]], s=100, color='red', marker='x', 
                   label='End', zorder=5)
axes[1, 1].scatter([target_height], [0], s=200, color='orange', marker='*', 
                   label='Target', zorder=5)
axes[1, 1].set_xlabel('Position (m)')
axes[1, 1].set_ylabel('Velocity (m/s)')
axes[1, 1].set_title('LQR: Phase Portrait', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ LQR simulation complete")

## Section 7: LSTM Inverse Model (Controller Learning)

In [ ]:
# Build inverse model: command = f(desired_acceleration_history)
# This trains a neural network controller: u = f(historical accelerations)
# We swap the roles of input and output

inverse_model = Sequential([
    LSTM(32, input_shape=(n_timesteps, 1), return_sequences=True, activation='relu'),
    LSTM(16, return_sequences=True, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for normalized commands [0,1]
])

inverse_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("Inverse Model (Controller) Architecture:")
inverse_model.summary()

# Prepare data: swap X and y roles
# Now X_train becomes desired accelerations (targets)
# And y_train becomes the commands (inputs we should generate)
print("\n" + "="*60)
print("Training Inverse LSTM Model (Neural Network Controller)...")
print("="*60)

history_inverse = inverse_model.fit(
    y_train, X_train,  # Swapped: learn command from desired acceleration
    validation_data=(y_test, X_test),
    epochs=25,
    batch_size=16,
    verbose=1
)

# Evaluate
test_loss_inv = inverse_model.evaluate(y_test, X_test, verbose=0)
print(f"\nInverse Model Test Loss: {test_loss_inv:.6f}")

In [ ]:
# Plot inverse model training history
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

axes[0].plot(history_inverse.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history_inverse.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Inverse Model (Controller): Training History', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

axes[1].plot(history_inverse.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history_inverse.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Inverse Model: Mean Absolute Error', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Inverse model training complete")

## Section 8: Closed-Loop Simulation with Forward & Inverse Models

In [ ]:
# Simulate closed-loop control with LSTM forward+inverse models
# We'll use the inverse model to compute commands from desired accelerations

def simulate_lstm_controller(target_height, sim_duration, forward_model, inverse_model, dt_sim=0.05):
    """
    Simulate closed-loop control using:
    1. Inverse model to compute command from desired trajectory
    2. Forward model to predict actual acceleration
    """
    num_steps = int(sim_duration / dt_sim)
    
    # Initialize state
    pos = 0.0
    vel = 0.0
    
    # Storage
    time_arr = np.linspace(0, sim_duration, num_steps)
    pos_hist = np.zeros(num_steps)
    vel_hist = np.zeros(num_steps)
    acc_hist = np.zeros(num_steps)
    cmd_hist = np.zeros(num_steps)
    
    # Time window for LSTM (use past history)
    history_window = min(100, n_timesteps)
    past_commands = np.zeros((1, history_window, 1))
    
    for i in range(num_steps):
        # Desired trajectory (ramp to target, then maintain)
        time_reach = 10.0  # Seconds to reach target
        if i * dt_sim < time_reach:
            target_pos = (i * dt_sim / time_reach) * target_height
            target_vel = target_height / time_reach
        else:
            target_pos = target_height
            target_vel = 0.0
        
        # Simple proportional + derivative control for desired acceleration
        k_p = 2.0
        k_d = 1.0
        pos_error = target_pos - pos
        vel_error = target_vel - vel
        desired_accel = k_p * pos_error + k_d * vel_error
        
        # Normalize desired acceleration for input to inverse model
        desired_accel_norm = (desired_accel / (global_max_abs_y * 2)) * 0.5 + 0.5  # Scale to ~[0,1]
        desired_accel_norm = np.clip(desired_accel_norm, 0, 1)
        
        # Use inverse model to get command
        desired_input = np.array([[desired_accel_norm]]).reshape(1, 1, 1)
        command = inverse_model.predict(desired_input, verbose=0)[0, 0, 0]
        command = np.clip(command, 0, 1)
        
        # Update history
        past_commands = np.roll(past_commands, -1, axis=1)
        past_commands[0, -1, 0] = command
        
        # Use forward model to predict acceleration
        predicted_accel_norm = forward_model.predict(past_commands, verbose=0)[0, -1, 0]
        actual_accel = predicted_accel_norm * global_max_abs_y
        
        # Update state with simple integration
        vel = vel + actual_accel * dt_sim
        pos = pos + vel * dt_sim
        
        # Store
        pos_hist[i] = pos
        vel_hist[i] = vel
        acc_hist[i] = actual_accel
        cmd_hist[i] = command
    
    return time_arr, pos_hist, vel_hist, acc_hist, cmd_hist

# Run LSTM controller simulation
target_height_lstm = 10.0
sim_duration_lstm = 20.0

time_lstm, pos_lstm, vel_lstm, acc_lstm, cmd_lstm = simulate_lstm_controller(
    target_height_lstm, sim_duration_lstm, forward_model, inverse_model, dt_sim=0.05
)

print(f"LSTM Controller Simulation:")
print(f"  Final position: {pos_lstm[-1]:.4f} m (target: {target_height_lstm} m)")
print(f"  Final velocity: {vel_lstm[-1]:.4f} m/s")
print(f"  Max acceleration: {np.max(np.abs(acc_lstm)):.4f} m/s²")

In [ ]:
# Plot LSTM controller simulation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Position
axes[0, 0].plot(time_lstm, pos_lstm, linewidth=2.5, color='darkgreen', label='LSTM Position')
axes[0, 0].plot(time_lqr, pos_lqr, linewidth=2, color='lightgreen', label='LQR Position', alpha=0.7, linestyle='--')
axes[0, 0].axhline(y=target_height_lstm, color='red', linestyle='--', linewidth=2, label='Target')
axes[0, 0].set_ylabel('Position (m)')
axes[0, 0].set_title('Comparison: Position Trajectory', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Velocity
axes[0, 1].plot(time_lstm, vel_lstm, linewidth=2.5, color='darkblue', label='LSTM Velocity')
axes[0, 1].plot(time_lqr, vel_lqr, linewidth=2, color='lightblue', label='LQR Velocity', alpha=0.7, linestyle='--')
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0, 1].set_ylabel('Velocity (m/s)')
axes[0, 1].set_title('Comparison: Velocity Trajectory', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Acceleration / Control
axes[1, 0].plot(time_lstm, acc_lstm, linewidth=2.5, color='darkviolet', label='LSTM Acceleration')
axes[1, 0].plot(time_lqr, acc_lqr, linewidth=2, color='plum', label='LQR Acceleration', alpha=0.7, linestyle='--')
axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[1, 0].set_ylabel('Acceleration (m/s²)')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_title('Comparison: Control Input', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Commands
axes[1, 1].plot(time_lstm, cmd_lstm, linewidth=2.5, color='darkorange')
axes[1, 1].fill_between(time_lstm, cmd_lstm, alpha=0.3, color='darkorange')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Normalized Command')
axes[1, 1].set_title('LSTM Controller: Command Output', fontweight='bold')
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ LSTM controller simulation complete")

## Section 9: Comprehensive Performance Comparison

### Comparison Metrics:
1. **Tracking Error**: How close actual position is to target
2. **Velocity Smoothness**: Variations in velocity (lower is better)
3. **Control Effort**: Total energy used (integral of squared control)
4. **Rise Time**: Time to reach 90% of target
5. **Overshoot**: How much the system exceeds target

In [ ]:
# Calculate performance metrics
def compute_metrics(time, position, velocity, control, target_pos, controller_name):
    """Compute control performance metrics"""
    
    # Tracking error
    pos_error = np.abs(position - target_pos)
    mean_pos_error = np.mean(pos_error)
    max_pos_error = np.max(pos_error)
    
    # Velocity smoothness (derivative)
    vel_smoothness = np.mean(np.abs(np.diff(velocity)))
    
    # Control effort
    control_effort = np.sum(control**2) * (time[1] - time[0])
    
    # Rise time (time to reach 90% of target)
    target_90 = 0.9 * target_pos
    indices_reached = np.where(position >= target_90)[0]
    if len(indices_reached) > 0:
        rise_time = time[indices_reached[0]]
    else:
        rise_time = np.inf
    
    # Overshoot
    max_pos = np.max(position)
    overshoot = max(0, max_pos - target_pos)
    overshoot_percent = (overshoot / target_pos * 100) if target_pos != 0 else 0
    
    # Final state error
    final_pos_error = np.abs(position[-1] - target_pos)
    final_vel_error = np.abs(velocity[-1])
    
    metrics = {
        'Controller': controller_name,
        'Mean Pos Error (m)': mean_pos_error,
        'Max Pos Error (m)': max_pos_error,
        'Final Pos Error (m)': final_pos_error,
        'Rise Time (s)': rise_time,
        'Overshoot (m)': overshoot,
        'Overshoot (%)': overshoot_percent,
        'Velocity Smoothness': vel_smoothness,
        'Control Effort': control_effort,
        'Final Velocity (m/s)': final_vel_error,
    }
    
    return metrics

# Compute metrics for both controllers
metrics_lqr = compute_metrics(time_lqr, pos_lqr, vel_lqr, control_lqr, target_height, 'LQR')
metrics_lstm = compute_metrics(time_lstm, pos_lstm, vel_lstm, cmd_lstm, target_height_lstm, 'LSTM')

# Display comparison table
import pandas as pd
metrics_df = pd.DataFrame([metrics_lqr, metrics_lstm])
print("\n" + "="*100)
print("PERFORMANCE COMPARISON: LQR vs LSTM Controller")
print("="*100)
print(metrics_df.to_string(index=False))
print("="*100)

In [ ]:
# Detailed comparison visualization
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)

# Row 1: Position and errors
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(time_lqr, pos_lqr, label='LQR', linewidth=2.5, color='green')
ax1.plot(time_lstm, pos_lstm, label='LSTM', linewidth=2.5, color='purple', alpha=0.8)
ax1.axhline(y=target_height, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Target')
ax1.set_ylabel('Position (m)')
ax1.set_title('Position Tracking', fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

ax2 = fig.add_subplot(gs[0, 1])
pos_error_lqr = np.abs(pos_lqr - target_height)
pos_error_lstm = np.abs(pos_lstm - target_height_lstm)
ax2.semilogy(time_lqr, pos_error_lqr, label='LQR', linewidth=2, color='green')
ax2.semilogy(time_lstm, pos_error_lstm, label='LSTM', linewidth=2, color='purple', alpha=0.8)
ax2.set_ylabel('Absolute Error (m)')
ax2.set_title('Position Tracking Error (log scale)', fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3, which='both')

ax3 = fig.add_subplot(gs[0, 2])
ax3.plot(time_lqr, pos_lqr - target_height, label='LQR', linewidth=2, color='green', alpha=0.7)
ax3.plot(time_lstm, pos_lstm - target_height_lstm, label='LSTM', linewidth=2, color='purple', alpha=0.7)
ax3.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax3.fill_between(time_lqr, 0, pos_lqr - target_height, alpha=0.2, color='green')
ax3.fill_between(time_lstm, 0, pos_lstm - target_height_lstm, alpha=0.2, color='purple')
ax3.set_ylabel('Position Error (m)')
ax3.set_title('Position Deviation from Target', fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Row 2: Velocity and control input
ax4 = fig.add_subplot(gs[1, 0])
ax4.plot(time_lqr, vel_lqr, label='LQR', linewidth=2.5, color='blue')
ax4.plot(time_lstm, vel_lstm, label='LSTM', linewidth=2.5, color='cyan', alpha=0.8)
ax4.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax4.set_ylabel('Velocity (m/s)')
ax4.set_title('Velocity Trajectory', fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

ax5 = fig.add_subplot(gs[1, 1])
ax5.plot(time_lqr, acc_lqr, label='LQR', linewidth=2.5, color='darkviolet')
ax5.plot(time_lstm, acc_lstm, label='LSTM', linewidth=2.5, color='orange', alpha=0.7)
ax5.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax5.set_ylabel('Acceleration (m/s²)')
ax5.set_title('Control Input (Acceleration)', fontweight='bold')
ax5.legend(fontsize=9)
ax5.grid(True, alpha=0.3)

ax6 = fig.add_subplot(gs[1, 2])
ax6.plot(time_lqr, np.cumsum(acc_lqr**2) * (time_lqr[1] - time_lqr[0]), 
         label='LQR', linewidth=2.5, color='darkviolet')
ax6.plot(time_lstm, np.cumsum(cmd_lstm**2) * (time_lstm[1] - time_lstm[0]), 
         label='LSTM', linewidth=2.5, color='orange', alpha=0.7)
ax6.set_xlabel('Time (s)')
ax6.set_ylabel('Cumulative Energy')
ax6.set_title('Control Energy (Cumulative)', fontweight='bold')
ax6.legend(fontsize=9)
ax6.grid(True, alpha=0.3)

# Row 3: Phase portrait and metrics summary
ax7 = fig.add_subplot(gs[2, 0])
ax7.plot(pos_lqr, vel_lqr, label='LQR', linewidth=2.5, color='green', alpha=0.7)
ax7.plot(pos_lstm, vel_lstm, label='LSTM', linewidth=2.5, color='purple', alpha=0.7)
ax7.scatter([pos_lqr[0]], [vel_lqr[0]], s=80, marker='o', color='green', zorder=5)
ax7.scatter([pos_lstm[0]], [vel_lstm[0]], s=80, marker='o', color='purple', zorder=5)
ax7.scatter([pos_lqr[-1]], [vel_lqr[-1]], s=100, marker='x', color='darkgreen', linewidth=3, zorder=5)
ax7.scatter([pos_lstm[-1]], [vel_lstm[-1]], s=100, marker='x', color='darkviolet', linewidth=3, zorder=5)
ax7.scatter([target_height], [0], s=200, marker='*', color='red', zorder=5, label='Target')
ax7.set_xlabel('Position (m)')
ax7.set_ylabel('Velocity (m/s)')
ax7.set_title('Phase Portrait', fontweight='bold')
ax7.legend(fontsize=9)
ax7.grid(True, alpha=0.3)

# Metrics comparison bar chart
ax8 = fig.add_subplot(gs[2, 1:])
metrics_names = ['Mean Pos\nError', 'Rise Time\n(s)', 'Overshoot\n(%)', 'Velocity\nSmoothing', 'Control\nEffort']
lqr_vals = [
    metrics_lqr['Mean Pos Error (m)'],
    metrics_lqr['Rise Time (s)'] / 10,  # Normalized
    metrics_lqr['Overshoot (%)'],
    metrics_lqr['Velocity Smoothness'],
    metrics_lqr['Control Effort'] / 1000  # Normalized
]
lstm_vals = [
    metrics_lstm['Mean Pos Error (m)'],
    metrics_lstm['Rise Time (s)'] / 10,  # Normalized
    metrics_lstm['Overshoot (%)'],
    metrics_lstm['Velocity Smoothness'],
    metrics_lstm['Control Effort'] / 1000  # Normalized
]

x = np.arange(len(metrics_names))
width = 0.35

bars1 = ax8.bar(x - width/2, lqr_vals, width, label='LQR', color='green', alpha=0.7)
bars2 = ax8.bar(x + width/2, lstm_vals, width, label='LSTM', color='purple', alpha=0.7)

ax8.set_ylabel('Normalized Value')
ax8.set_title('Performance Metrics Comparison (Normalized)', fontweight='bold')
ax8.set_xticks(x)
ax8.set_xticklabels(metrics_names, fontsize=9)
ax8.legend()
ax8.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax8.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}', ha='center', va='bottom', fontsize=8)

plt.show()

print("\n✓ Comprehensive comparison complete")

## Section 10: Summary and Analysis

### Key Findings:

**LQR Controller:**
- Based on linear state-space model
- Optimal gain computed analytically via DARE
- Fast rise time with minimal overshoot
- Smooth control inputs
- Robust to linear perturbations
- Limited to linear systems

**LSTM-Based Controller:**
- Data-driven approach learning from experimental data
- Can capture nonlinear system dynamics
- No need for manual system identification
- Can generalize to similar operating conditions
- Computational cost: real-time neural network evaluation
- May require more data for generalization

### Recommendations:
1. **Use LQR** when: System is approximately linear, safety is critical, computational resources are limited
2. **Use LSTM** when: System is nonlinear, abundant data is available, real-time requirements are moderate
3. **Hybrid approach**: Use LSTM for trajectory planning and LQR for stabilization

## BONUS Section: System Identification - Estimating A and B from Data

**Why identify A and B?**

The matrices I used initially were **generic kinematic equations**, assuming:
- Perfect integration: `x(k+1) = x(k) + v(k)·dt`
- Perfect velocity update: `v(k+1) = v(k) + a(k)·dt`

**But your actual system may have:**
- Actuator dynamics (delays, nonlinearities)
- Friction/damping
- Nonlinear thrust response
- Inertial effects

This section uses **Least Squares System Identification** to extract the true A and B matrices from your experimental data.

In [ ]:
# ============================================================================
# SYSTEM IDENTIFICATION: Extract A and B matrices from experimental data
# ============================================================================
# 
# We want to identify a discrete linear system:
#   x(k+1) = A·x(k) + B·u(k)
#
# Where:
#   x(k) = [position, velocity]ᵀ
#   u(k) = normalized command input
#   A = state transition matrix (2x2)
#   B = input matrix (2x1)
#
# Method: Least Squares Identification
# ============================================================================

print("\n" + "="*80)
print("SYSTEM IDENTIFICATION FROM EXPERIMENTAL DATA")
print("="*80)

# Prepare data for system identification
# We need to double-integrate acceleration to get position and velocity

sample_idx = 0  # Use first sample for identification
command_sequence = X_train[sample_idx, :, 0]  # Normalized commands
acceleration_sequence = y_train[sample_idx, :, 0] * global_max_abs_y  # Denormalized acceleration

# Integrate to get velocity and position
velocity_sequence = np.cumsum(acceleration_sequence) * dt
position_sequence = np.cumsum(velocity_sequence) * dt

print(f"\nUsing sample {sample_idx} for identification:")
print(f"  Command length: {len(command_sequence)}")
print(f"  Acceleration: min={np.min(acceleration_sequence):.4f}, max={np.max(acceleration_sequence):.4f} m/s²")
print(f"  Velocity: min={np.min(velocity_sequence):.4f}, max={np.max(velocity_sequence):.4f} m/s")
print(f"  Position: min={np.min(position_sequence):.4f}, max={np.max(position_sequence):.4f} m")

# Build regression matrices for least squares
# Stack multiple time instances to get overdetermined system
n_data_points = len(command_sequence) - 1

# State at time k: [position(k), velocity(k)]ᵀ
X_k = np.column_stack([position_sequence[:-1], velocity_sequence[:-1]])  # (n_data_points, 2)

# State at time k+1
X_kp1 = np.column_stack([position_sequence[1:], velocity_sequence[1:]])  # (n_data_points, 2)

# Input at time k
U_k = command_sequence[:-1].reshape(-1, 1)  # (n_data_points, 1)

# Build augmented matrix: [X(k) | U(k)]
Phi = np.column_stack([X_k, U_k])  # (n_data_points, 3)

# Solve least squares: X(k+1) = Phi * theta
# where theta = [A11, A12, B1; A21, A22, B2]ᵀ

theta, residuals, rank, s = np.linalg.lstsq(Phi, X_kp1, rcond=None)

# Extract identified matrices
A_identified = theta[:2, :2].T  # Shape: (2, 2) - Note: theta is (3, 2)
B_identified = theta[2:3, :].T   # Shape: (2, 1)

# Properly reshape theta for 2x2 A and 2x1 B
# theta should be interpreted as columns of the result
A_identified = theta[:, :2].T  # First 2 rows (for each state equation)
B_identified = theta[:, 2:3].T  # Last row

# Actually, let's be more careful:
# For each state component, we have: x_i(k+1) = A_i · [x1(k), x2(k)] + B_i · u(k)
# This is 2 equations (for 2 state components)

A_identified = theta[:2, :].T  # theta has shape (3, 2), take first 2 rows, transpose
B_identified = theta[2, :].reshape(-1, 1)  # Take last row

print("\n" + "-"*80)
print("IDENTIFIED STATE-SPACE MATRICES (from Least Squares)")
print("-"*80)
print(f"\nA_identified (State Transition):\n{A_identified}")
print(f"\nB_identified (Input Matrix):\n{B_identified}")

# For comparison, print the theoretical matrices
print("\n" + "-"*80)
print("THEORETICAL KINEMATIC MATRICES (for reference)")
print("-"*80)
print(f"\nA_theoretical:\n{A_lqr}")
print(f"\nB_theoretical:\n{B_lqr}")

# Calculate identification error
residual_error = np.mean(residuals) if len(residuals) > 0 else 0
print(f"\nIdentification Residual Error: {residual_error:.6e}")

# Validate identification by predicting future states
print("\n" + "-"*80)
print("VALIDATION: Compare Identified Model vs Actual Data")
print("-"*80)

# Simulate with identified model
x_sim_identified = np.zeros((len(command_sequence), 2))
x_sim_identified[0] = [position_sequence[0], velocity_sequence[0]]

x_actual = np.column_stack([position_sequence, velocity_sequence])

for k in range(len(command_sequence) - 1):
    x_sim_identified[k+1] = A_identified @ x_sim_identified[k] + B_identified.flatten() * U_k[k, 0]

# Calculate prediction errors
pos_pred_error = np.mean(np.abs(x_sim_identified[:, 0] - position_sequence))
vel_pred_error = np.mean(np.abs(x_sim_identified[:, 1] - velocity_sequence))

print(f"\nMean Absolute Prediction Error:")
print(f"  Position: {pos_pred_error:.6f} m")
print(f"  Velocity: {vel_pred_error:.6f} m/s")

# Visualization comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Acceleration data
axes[0, 0].plot(acceleration_sequence, label='Measured', linewidth=2, color='blue')
axes[0, 0].fill_between(range(len(acceleration_sequence)), acceleration_sequence, alpha=0.3, color='blue')
axes[0, 0].set_ylabel('Acceleration (m/s²)')
axes[0, 0].set_title('Raw Acceleration Data', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Integrated velocity
axes[0, 1].plot(velocity_sequence, label='Measured', linewidth=2, color='green')
axes[0, 1].plot(x_sim_identified[:, 1], label='Identified Model', linewidth=2, 
               color='red', alpha=0.7, linestyle='--')
axes[0, 1].set_ylabel('Velocity (m/s)')
axes[0, 1].set_title('Velocity: Measured vs Identified Model', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Integrated position
axes[1, 0].plot(position_sequence, label='Measured', linewidth=2, color='purple')
axes[1, 0].plot(x_sim_identified[:, 0], label='Identified Model', linewidth=2, 
               color='orange', alpha=0.7, linestyle='--')
axes[1, 0].set_ylabel('Position (m)')
axes[1, 0].set_title('Position: Measured vs Identified Model', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Phase portrait
axes[1, 1].plot(position_sequence, velocity_sequence, label='Measured', linewidth=2, color='green')
axes[1, 1].plot(x_sim_identified[:, 0], x_sim_identified[:, 1], label='Identified Model', 
               linewidth=2, color='red', alpha=0.7, linestyle='--')
axes[1, 1].scatter([position_sequence[0]], [velocity_sequence[0]], s=100, marker='o', 
                  color='green', zorder=5, label='Start')
axes[1, 1].scatter([position_sequence[-1]], [velocity_sequence[-1]], s=100, marker='x', 
                  color='green', zorder=5)
axes[1, 1].set_xlabel('Position (m)')
axes[1, 1].set_ylabel('Velocity (m/s)')
axes[1, 1].set_title('Phase Portrait: Measured vs Identified Model', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\n✓ System identification complete")

In [ ]:
# ============================================================================
# LQR CONTROLLER WITH IDENTIFIED MATRICES
# ============================================================================

print("\n" + "="*80)
print("LQR CONTROLLER DESIGN WITH IDENTIFIED SYSTEM")
print("="*80)

# Solve DARE with identified matrices
try:
    P_identified = scipy.linalg.solve_discrete_are(A_identified, B_identified, Q_lqr, R_lqr)
    K_identified = np.linalg.inv(R_lqr + B_identified.T @ P_identified @ B_identified) @ \
                   (B_identified.T @ P_identified @ A_identified)
    
    print("\nIdentified System LQR Gain:")
    print(f"K_identified:\n{K_identified}")
    print(f"\nComparison with Theoretical Gain:")
    print(f"K_theoretical:\n{K_lqr}")
    print(f"\nGain Difference: {np.linalg.norm(K_identified - K_lqr):.6f}")
    
    # Simulate with identified matrices
    state_identified = np.array([[0.0], [0.0]])
    target_state = np.array([[target_height], [0.0]])
    
    pos_identified = np.zeros(num_steps_sim)
    vel_identified = np.zeros(num_steps_sim)
    acc_identified = np.zeros(num_steps_sim)
    
    for i in range(num_steps_sim):
        error_state = state_identified - target_state
        u_identified = -K_identified @ error_state
        acc_identified[i] = u_identified[0, 0]
        
        pos_identified[i] = state_identified[0, 0]
        vel_identified[i] = state_identified[1, 0]
        
        state_identified = A_identified @ state_identified + B_identified @ u_identified
    
    print(f"\nIdentified System LQR Performance:")
    print(f"  Final position: {pos_identified[-1]:.4f} m (target: {target_height} m)")
    print(f"  Final velocity: {vel_identified[-1]:.4f} m/s")
    print(f"  Max acceleration: {np.max(np.abs(acc_identified)):.4f} m/s²")
    
except Exception as e:
    print(f"Warning: Could not compute LQR for identified system: {e}")
    print("This may occur if the system is not controllable.")
    pos_identified = pos_lqr.copy()
    vel_identified = vel_lqr.copy()
    acc_identified = acc_lqr.copy()

# Comparison plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Position comparison
axes[0, 0].plot(time_lqr, pos_lqr, label='LQR (Theoretical)', linewidth=2.5, color='green', alpha=0.7)
axes[0, 0].plot(time_lqr, pos_identified, label='LQR (Identified)', linewidth=2.5, 
               color='darkgreen', alpha=0.7, linestyle='--')
axes[0, 0].axhline(y=target_height, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Target')
axes[0, 0].set_ylabel('Position (m)')
axes[0, 0].set_title('Position: Theoretical vs Identified Matrices', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Velocity comparison
axes[0, 1].plot(time_lqr, vel_lqr, label='LQR (Theoretical)', linewidth=2.5, color='blue', alpha=0.7)
axes[0, 1].plot(time_lqr, vel_identified, label='LQR (Identified)', linewidth=2.5, 
               color='darkblue', alpha=0.7, linestyle='--')
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[0, 1].set_ylabel('Velocity (m/s)')
axes[0, 1].set_title('Velocity: Theoretical vs Identified Matrices', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Control input comparison
axes[1, 0].plot(time_lqr, acc_lqr, label='LQR (Theoretical)', linewidth=2.5, color='purple', alpha=0.7)
axes[1, 0].plot(time_lqr, acc_identified, label='LQR (Identified)', linewidth=2.5, 
               color='darkviolet', alpha=0.7, linestyle='--')
axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Acceleration (m/s²)')
axes[1, 0].set_title('Control Input: Theoretical vs Identified', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Matrix comparison visualization
ax4 = axes[1, 1]
ax4.axis('off')

matrix_text = f"""
SYSTEM MATRICES COMPARISON
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Theoretical Matrices:
A_theoretical = 
  [{A_lqr[0,0]:.6f}  {A_lqr[0,1]:.6f}]
  [{A_lqr[1,0]:.6f}  {A_lqr[1,1]:.6f}]

B_theoretical = 
  [{B_lqr[0,0]:.6f}]
  [{B_lqr[1,0]:.6f}]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Identified Matrices:
A_identified = 
  [{A_identified[0,0]:.6f}  {A_identified[0,1]:.6f}]
  [{A_identified[1,0]:.6f}  {A_identified[1,1]:.6f}]

B_identified = 
  [{B_identified[0,0]:.6f}]
  [{B_identified[1,0]:.6f}]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

System Quality Indicator:
✓ If differences are small: 
  Theoretical model matches system

✗ If differences are large:
  System has additional dynamics
  (damping, friction, nonlinearity)
"""

ax4.text(0.05, 0.95, matrix_text, transform=ax4.transAxes, fontfamily='monospace',
         verticalalignment='top', fontsize=9, bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n✓ Comparison complete")

## Final Summary: Why Matrices Matter

### Three Approaches to Get A and B:

**1. Theoretical/Kinematic (What I initially used)** ❌ Arbitrary
```
A = [1    dt  ]    B = [0.5*dt²]
    [0    1   ]        [dt     ]
```
- ✅ Simple, interpretable
- ❌ Only works if system perfectly follows kinematics
- ❌ Ignores actuator dynamics, friction, delays

**2. System Identification (BEST) ✅ Data-driven**
```
From your experimental data, estimate:
  min ||X(k+1) - [A·X(k) + B·U(k)]||²
```
- ✅ Captures real system behavior
- ✅ Accounts for actuator nonlinearities
- ✅ Minimal assumptions
- ✓ Validated against your actual data

**3. Physics-Based Modeling** 🔬 (If available)
- Measure inertia, friction coefficients
- Determine actuator response curves
- Model aerodynamic effects

### What Your Identification Shows:

If `A_identified ≈ A_theoretical`:
- ✅ Your system behaves linearly
- ✅ Kinematic assumptions are valid
- ✅ LQR with theoretical matrices should work well

If `A_identified ≠ A_theoretical`:
- ⚠️ System has nonlinear dynamics
- ⚠️ May need higher-order model
- ⚠️ LSTM approach becomes more attractive

In [ ]:
# Final summary visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# Controller architecture comparison
ax1 = fig.add_subplot(gs[0, 0])
ax1.axis('off')
lqr_text = """
LQR (Linear Quadratic Regulator)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Approach: Classical Control
Model: Discrete Linear State-Space
Optimization: DARE (Riccati Equation)
Computation: Closed-form solution
Online Cost: Matrix multiplication O(n²)

Architecture:
  System Dynamics: Linear
  Control: u = -K(x - x_target)
  State: [position, velocity]
  
Advantages:
  ✓ Guaranteed stability
  ✓ Optimal for linear systems
  ✓ Low computational cost
  ✓ Interpretable
  
Disadvantages:
  ✗ Limited to linear systems
  ✗ Requires system identification
  ✗ May not capture nonlinearities
"""
ax1.text(0.05, 0.95, lqr_text, transform=ax1.transAxes, fontfamily='monospace',
         verticalalignment='top', fontsize=9, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))

ax2 = fig.add_subplot(gs[0, 1])
ax2.axis('off')
lstm_text = """
LSTM Neural Network Controller
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Approach: Data-Driven (ML)
Model: LSTM + Dense Layers
Optimization: Backpropagation
Computation: Learned from data
Online Cost: Neural net inference

Architecture:
  Forward Model: Command → Accel
  Inverse Model: Accel → Command
  Control: Learned by LSTM network
  State: Historical sequences
  
Advantages:
  ✓ Captures nonlinearities
  ✓ No manual identification needed
  ✓ Learns from data patterns
  ✓ Can generalize
  
Disadvantages:
  ✗ Requires training data
  ✗ Computational overhead
  ✗ Less interpretable
  ✗ May overfit
"""
ax2.text(0.05, 0.95, lstm_text, transform=ax2.transAxes, fontfamily='monospace',
         verticalalignment='top', fontsize=9, bbox=dict(boxstyle='round', facecolor='mediumpurple', alpha=0.3))

ax3 = fig.add_subplot(gs[0, 2])
ax3.axis('off')
comparison_text = f"""
Quantitative Comparison
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Metric                  LQR        LSTM
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Rise Time           {metrics_lqr['Rise Time (s)']:6.2f}s   {metrics_lstm['Rise Time (s)']:6.2f}s
Mean Pos Error      {metrics_lqr['Mean Pos Error (m)']:6.4f}m   {metrics_lstm['Mean Pos Error (m)']:6.4f}m
Max Overshoot       {metrics_lqr['Overshoot (%)']:6.2f}%   {metrics_lstm['Overshoot (%)']:6.2f}%
Final Pos Error     {metrics_lqr['Final Pos Error (m)']:6.4f}m   {metrics_lstm['Final Pos Error (m)']:6.4f}m
Velocity Smooth     {metrics_lqr['Velocity Smoothness']:6.4f}   {metrics_lstm['Velocity Smoothness']:6.4f}
Control Energy      {metrics_lqr['Control Effort']:6.2f}   {metrics_lstm['Control Effort']:6.2f}

Key Observation:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
LQR provides optimal linear control
with guaranteed convergence.

LSTM learns data patterns but may
require tuning for best performance.
"""
ax3.text(0.05, 0.95, comparison_text, transform=ax3.transAxes, fontfamily='monospace',
         verticalalignment='top', fontsize=8.5, bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))

# Bottom row: Decision tree
ax4 = fig.add_subplot(gs[1, :])
ax4.axis('off')

decision_text = """
DECISION FRAMEWORK: Which Controller to Use?
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

                                        ┌─── System Well-Characterized? ───┐
                                        │                                  │
                                     YES                               NO (or High uncertainty)
                                        │                                  │
                        ┌───────────────┴──────────┐            ┌─────────┴──────────────┐
                        │                          │            │                        │
                    Linear System?           Nonlinear System?   Enough Data Available?  │
                        │                          │            │                        │
                    ╔═══╩═══╗                  ╔═══╩═══╗        │                        │
                    ║ LQR ✓ ║                  ║ LSTM  ║    YES │                    NO │
                    ║    ✓  ║ OPTIMAL          ║ Hybrid║        │                        │
                    ╚═══════╝                  ╚═══════╝        │                        │
                    Fast, Proven                 Good if        │                        │
                    Guaranteed Stability         data/compute   │                        │
                                                available      ╔══════════════════╗  ╔═════════════════╗
                    Safety Critical?                           ║ LQR or PID      ║  ║ Collect Data or ║
                         ║                                     ║ + Learning      ║  ║ Model Externally║
                    ╔════╩════╗                                ╚══════════════════╝  ╚═════════════════╝
                    ║  LQR    ║
                    ║ PREFERRED║
                    ╚═════════╝
                    Predictable, 
                    Analyzable

HYBRID STRATEGY: LSTM for Planning + LQR for Stabilization
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Reference Trajectory                  LSTM Trajectory             LQR Stabilization          Final Trajectory
  (Target Path)          ───────→       Generator (Plan)   ─────→   (Stabilize around)  ─────→  (Smooth, Stable)
                                        
                         Data-Driven    Provides rough plan          Classical control    Guaranteed convergence
                         approach       Good for complex paths       Linear corrections   + Adaptability
"""

ax4.text(0.02, 0.98, decision_text, transform=ax4.transAxes, fontfamily='monospace',
         verticalalignment='top', fontsize=8.5, bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.4))

plt.suptitle('State-Space Control vs LSTM-Based Control: Complete Analysis', 
             fontsize=14, fontweight='bold', y=0.98)
plt.show()

print("\n" + "="*100)
print("ANALYSIS COMPLETE")
print("="*100)
print(f"\nNotebook generated: {len([1 for _ in [1,2,3,4,5,6,7,8,9,10]])} sections with comprehensive plots and analysis")
print("\nKey Outputs:")
print("  1. Forward LSTM Model: System dynamics learning")
print("  2. LQR Controller: Analytical optimal control")
print("  3. LSTM Controller: Data-driven control")
print("  4. Performance Metrics: Quantitative comparison")
print("  5. Visualization: 15+ comparison plots")
print("  6. Decision Framework: Guidance for controller selection")
print("="*100)

# ============================================================================
## Section 11: State Space Models (SSM) for Sequence Modeling
## ============================================================================

### Theory: SSM Introduction

State Space Models (SSMs) are a powerful framework for modeling **sequential and temporal data**. Originally used in control theory, SSMs have been recently rediscovered for deep learning (S4, Mamba, etc.).

**Key Insight:** SSMs represent data as continuous dynamical systems that are discretized for computation.

### Continuous SSM Formulation

A continuous SSM is defined by four learnable matrices and two equations:

$$\boxed{\begin{align}
x'(t) &= Ax(t) + Bu(t) \quad \text{(State equation)} \\
y(t) &= Cx(t) + Du(t) \quad \text{(Output equation)}
\end{align}}$$

**Where:**
- $x(t) \in \mathbb{R}^n$ : Hidden state (dimension $n$)
- $u(t) \in \mathbb{R}^m$ : Input signal (dimension $m$)
- $y(t) \in \mathbb{R}^p$ : Output signal (dimension $p$)
- $A \in \mathbb{R}^{n \times n}$ : State transition matrix
- $B \in \mathbb{R}^{n \times m}$ : Input matrix
- $C \in \mathbb{R}^{p \times n}$ : Output matrix
- $D \in \mathbb{R}^{p \times m}$ : Feedthrough matrix (often set to 0)

**Simplified form** (removing $D$ as a skip connection):
$$\boxed{\begin{align}
x'(t) &= Ax(t) + Bu(t) \\
y(t) &= Cx(t)
\end{align}}$$

### Three Views of SSMs

SSMs have **three equivalent representations**:

1. **Continuous View** ✓ Handles continuous/irregular data
2. **Recursive View** ✓ Efficient inference (like RNNs)
3. **Convolutional View** ✓ Efficient training (parallelizable)

### Discretization: From Continuous to Discrete

To implement SSMs on computers, we must discretize the continuous equations. Using the **trapezoid/bilinear method**:

$$\Delta = t_{n+1} - t_n \quad \text{(sampling interval)}$$

$$x_n' = Ax_n + Bu_n$$

Apply trapezoid rule:
$$x_{n+1} - x_n = \frac{\Delta}{2}(Ax_n + Bu_n + Ax_{n+1} + Bu_{n+1})$$

Rearrange:
$$\boxed{x_{n+1} = \bar{A}x_n + \bar{B}u_n}$$

$$\boxed{y_n = \bar{C}x_n}$$

**Where the discretized matrices are:**

$$\boxed{\begin{align}
\bar{A} &= (I - \frac{\Delta}{2}A)^{-1}(I + \frac{\Delta}{2}A) \\
\bar{B} &= (I - \frac{\Delta}{2}A)^{-1} \Delta B \\
\bar{C} &= C
\end{align}}$$

### Recursive (RNN-like) View

Process sequences step-by-step:

$$\boxed{\begin{align}
x_k &= \bar{A}x_{k-1} + \bar{B}u_k \quad \text{(Recurrence)} \\
y_k &= \bar{C}x_k \quad \text{(Output)}
\end{align}}$$

Advantages:
- ✓ Efficient inference (constant-time updates)
- ✗ Slow training (not parallelizable)

### Convolutional (CNN-like) View

By unrolling the recurrence, we can express output as a convolution:

$$y_k = \bar{C}(\bar{A}^k\bar{B}u_0 + \bar{A}^{k-1}\bar{B}u_1 + \cdots + \bar{B}u_k)$$

$$\boxed{y_k = \sum_{j=0}^{k} \bar{K}_j u_{k-j}}$$

**Where $\bar{K}$ is the convolutional kernel:**

$$\boxed{\bar{K}_k = \bar{C}\bar{A}^k\bar{B}}$$

Advantages:
- ✓ Efficient training (parallelizable via FFT)
- ✗ Slow inference (must recalculate entire sequence)

### Comparison: Recursive vs Convolutional

| Aspect | Recursive | Convolutional |
|--------|-----------|--------------|
| **Training Speed** | Slow (sequential) | Fast (parallel) |
| **Inference Speed** | Fast (constant time) | Slow (full sequence) |
| **Memory** | $O(1)$ | $O(L)$ where $L$ = sequence length |
| **Best For** | Online/streaming | Batch processing |

**Practical Strategy:**
- **Train** using convolutional view (fast)
- **Infer** using recursive view (efficient)

### Matrix Initialization: HiPPO

Random initialization of $A$ leads to poor results. The **HiPPO (High-Order Polynomial Projection Operator)** matrix provides a principled initialization optimized for temporal dynamics.

Key properties:
- Captures Legendre polynomial dynamics
- Enables $O(1)$ memory for long sequences
- Diagonal or NPLR structure for efficiency

In [ ]:
# ============================================================================
# SSM IMPLEMENTATION FOR CONTROL SEQUENCES
# ============================================================================

print("\n" + "="*80)
print("STATE SPACE MODEL (SSM) IMPLEMENTATION")
print("="*80)

class SSMModel:
    """
    State Space Model for sequence modeling
    
    Continuous form:
        x'(t) = A·x(t) + B·u(t)
        y(t) = C·x(t)
    
    Discrete form (after trapezoid discretization):
        x_k = Ā·x_{k-1} + B̄·u_k
        y_k = C̄·x_k
    """
    
    def __init__(self, state_dim, input_dim, output_dim, dt=0.05, init_method='random'):
        """
        Initialize SSM with given dimensions
        
        Args:
            state_dim: Dimension of hidden state (n)
            input_dim: Dimension of input (m)
            output_dim: Dimension of output (p)
            dt: Discretization interval (Δ)
            init_method: 'random', 'hippopotamus' (HiPPO), or 'stable'
        """
        self.n = state_dim      # Hidden state dimension
        self.m = input_dim      # Input dimension
        self.p = output_dim     # Output dimension
        self.dt = dt            # Sampling time
        
        # Initialize continuous matrices
        if init_method == 'random':
            self.A = np.random.randn(self.n, self.n) * 0.1
        elif init_method == 'stable':
            # Ensure stable system (all eigenvalues have negative real part)
            A_rand = np.random.randn(self.n, self.n)
            self.A = A_rand - 2 * np.eye(self.n)  # Shift to negative region
        elif init_method == 'diagonal':
            # Diagonal matrix with negative eigenvalues
            self.A = np.diag(-np.exp(np.random.uniform(0, 2, self.n)))
        else:
            self.A = np.random.randn(self.n, self.n) * 0.1
        
        self.B = np.random.randn(self.n, self.m) * 0.1
        self.C = np.random.randn(self.p, self.n) * 0.1
        
        # Discretize using trapezoid method
        self._discretize()
        
    def _discretize(self):
        """
        Discretize continuous SSM using trapezoid (bilinear) method
        
        Ā = (I - Δ/2·A)^(-1) · (I + Δ/2·A)
        B̄ = (I - Δ/2·A)^(-1) · Δ·B
        C̄ = C
        """
        I = np.eye(self.n)
        
        # Factor: (I - Δ/2·A)
        left_factor = I - (self.dt / 2) * self.A
        right_factor = I + (self.dt / 2) * self.A
        
        # Compute A_bar with regularization for numerical stability
        try:
            left_inv = np.linalg.inv(left_factor)
            self.A_bar = left_inv @ right_factor
            self.B_bar = left_inv @ (self.dt * self.B)
            self.C_bar = self.C.copy()
        except np.linalg.LinAlgError:
            print("Warning: Singular matrix in discretization, using regularization")
            left_factor += 1e-6 * np.eye(self.n)
            left_inv = np.linalg.inv(left_factor)
            self.A_bar = left_inv @ right_factor
            self.B_bar = left_inv @ (self.dt * self.B)
            self.C_bar = self.C.copy()
    
    def forward_recursive(self, u_sequence):
        """
        Recursive (RNN-like) forward pass
        
        x_k = Ā·x_{k-1} + B̄·u_k
        y_k = C̄·x_k
        
        Args:
            u_sequence: (L, m) array of input sequence
        
        Returns:
            y_sequence: (L, p) array of output sequence
            x_states: (L+1, n) array of hidden states
        """
        L = u_sequence.shape[0]
        y_sequence = np.zeros((L, self.p))
        x_states = np.zeros((L + 1, self.n))
        
        for k in range(L):
            # Update state
            x_states[k + 1] = self.A_bar @ x_states[k] + self.B_bar @ u_sequence[k]
            # Compute output
            y_sequence[k] = self.C_bar @ x_states[k + 1]
        
        return y_sequence, x_states
    
    def get_convolution_kernel(self, length):
        """
        Compute convolutional kernel for SSM
        
        K̄_k = C̄·Ā^k·B̄
        
        Args:
            length: Length of sequence (L)
        
        Returns:
            kernel: (L, p, m) convolutional kernel
        """
        kernel = np.zeros((length, self.p, self.m))
        A_power = np.eye(self.n)
        
        for k in range(length):
            # K̄_k = C̄·Ā^k·B̄
            kernel[k] = self.C_bar @ A_power @ self.B_bar
            A_power = A_power @ self.A_bar
        
        return kernel
    
    def forward_convolutional(self, u_sequence):
        """
        Convolutional (CNN-like) forward pass
        
        y = K̄ * u (convolution)
        
        Args:
            u_sequence: (L, m) array of input sequence
        
        Returns:
            y_sequence: (L, p) array of output sequence
        """
        L = u_sequence.shape[0]
        kernel = self.get_convolution_kernel(L)
        
        y_sequence = np.zeros((L, self.p))
        for k in range(L):
            # y_k = sum_j K̄_j · u_{k-j}
            for j in range(k + 1):
                y_sequence[k] += kernel[j].T @ u_sequence[k - j]
        
        return y_sequence
    
    def get_statistics(self):
        """Get SSM statistics"""
        eigs_A = np.linalg.eigvals(self.A)
        eigs_A_bar = np.linalg.eigvals(self.A_bar)
        
        return {
            'A_stability': np.max(np.real(eigs_A)),
            'A_bar_stability': np.max(np.abs(eigs_A_bar)),
            'A_condition': np.linalg.cond(self.A),
        }


# ============================================================================
# Create and test SSM for drone control
# ============================================================================

print("\nCreating SSM for drone control...")
state_dim = 4       # State: [pos, vel, accel, hidden]
input_dim = 1       # Input: normalized command
output_dim = 1      # Output: acceleration

# Initialize with stable diagonal structure
ssm = SSMModel(state_dim, input_dim, output_dim, dt=dt, init_method='stable')

print(f"\nSSM Configuration:")
print(f"  State dimension (n): {state_dim}")
print(f"  Input dimension (m): {input_dim}")
print(f"  Output dimension (p): {output_dim}")
print(f"  Discretization interval (Δ): {dt}s")

print(f"\nContinuous Matrices (A, B, C):")
print(f"  A shape: {ssm.A.shape}, condition number: {np.linalg.cond(ssm.A):.2f}")
print(f"  B shape: {ssm.B.shape}")
print(f"  C shape: {ssm.C.shape}")

print(f"\nDiscretized Matrices (Ā, B̄, C̄):")
print(f"  Ā shape: {ssm.A_bar.shape}")
print(f"  B̄ shape: {ssm.B_bar.shape}")
print(f"  C̄ shape: {ssm.C_bar.shape}")

# Get statistics
stats = ssm.get_statistics()
print(f"\nSSM Stability Analysis:")
print(f"  Max eigenvalue (continuous A): {stats['A_stability']:.6f}")
print(f"  Max magnitude eigenvalue (discrete Ā): {stats['A_bar_stability']:.6f}")
if stats['A_bar_stability'] < 1.0:
    print(f"  ✓ Discrete system is STABLE (|λ| < 1)")
else:
    print(f"  ✗ Warning: Discrete system may be UNSTABLE (|λ| ≥ 1)")

print("\n✓ SSM created")

In [ ]:
# ============================================================================
# SSM TRAINING WITH LEARNABLE PARAMETERS
# ============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class SSMLayer(nn.Module):
    """Learnable SSM layer using PyTorch"""
    
    def __init__(self, state_dim, input_dim, output_dim, dt=0.05):
        super().__init__()
        self.n = state_dim
        self.m = input_dim
        self.p = output_dim
        self.dt = dt
        
        # Learnable continuous matrices
        self.A = nn.Parameter(torch.randn(state_dim, state_dim) * 0.1)
        self.B = nn.Parameter(torch.randn(state_dim, input_dim) * 0.1)
        self.C = nn.Parameter(torch.randn(output_dim, state_dim) * 0.1)
        
        # Initialize A to be stable
        with torch.no_grad():
            self.A.copy_(torch.randn(state_dim, state_dim) - 2 * torch.eye(state_dim))
    
    def discretize(self):
        """Discretize continuous SSM using trapezoid method"""
        I = torch.eye(self.n, device=self.A.device)
        
        left_factor = I - (self.dt / 2) * self.A
        right_factor = I + (self.dt / 2) * self.A
        
        # Add regularization for numerical stability
        left_factor_reg = left_factor + 1e-6 * I
        
        A_bar = torch.linalg.solve(left_factor_reg, right_factor)
        B_bar = torch.linalg.solve(left_factor_reg, self.dt * self.B)
        C_bar = self.C
        
        return A_bar, B_bar, C_bar
    
    def forward_recursive(self, u_sequence):
        """
        Recursive forward pass
        
        Args:
            u_sequence: (batch, length, input_dim) tensor
        
        Returns:
            y_sequence: (batch, length, output_dim) tensor
        """
        batch_size, length, _ = u_sequence.shape
        device = u_sequence.device
        
        A_bar, B_bar, C_bar = self.discretize()
        
        # Initialize hidden state
        x = torch.zeros(batch_size, self.n, device=device)
        y_sequence = []
        
        for k in range(length):
            # x_k = Ā·x_{k-1} + B̄·u_k
            x = (A_bar @ x.T).T + (u_sequence[:, k:k+1] @ B_bar.T)
            # y_k = C̄·x_k
            y = x @ C_bar.T
            y_sequence.append(y)
        
        y_sequence = torch.stack(y_sequence, dim=1)  # (batch, length, output_dim)
        return y_sequence
    
    def forward(self, u_sequence):
        """Forward pass (recursive view)"""
        return self.forward_recursive(u_sequence)


# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize SSM model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ssm_model = SSMLayer(state_dim=8, input_dim=1, output_dim=1, dt=dt).to(device)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(ssm_model.parameters(), lr=0.001)

print("\n" + "="*80)
print("TRAINING SSM MODEL")
print("="*80)
print(f"Device: {device}")
print(f"Batch size: {batch_size}")
print(f"Epochs: 25")

# Training loop
ssm_train_losses = []
ssm_val_losses = []
epochs = 25

for epoch in range(epochs):
    # Training
    train_loss = 0.0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        output = ssm_model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    ssm_train_losses.append(train_loss)
    
    # Validation
    with torch.no_grad():
        val_output = ssm_model(X_test_tensor.to(device))
        val_loss = criterion(val_output, y_test_tensor.to(device))
        ssm_val_losses.append(val_loss.item())
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:2d}/{epochs} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss.item():.6f}")

print(f"\n✓ SSM training complete")
print(f"  Final train loss: {ssm_train_losses[-1]:.6f}")
print(f"  Final val loss: {ssm_val_losses[-1]:.6f}")

# Get predictions on test set
ssm_model.eval()
with torch.no_grad():
    y_pred_ssm_norm = ssm_model(X_test_tensor.to(device)).cpu().numpy()

# Denormalize predictions
y_pred_ssm = y_pred_ssm_norm * global_max_abs_y
y_test_real_check = y_test * global_max_abs_y

# Compute metrics
ssm_mse = np.mean((y_pred_ssm - y_test_real_check)**2)
ssm_rmse = np.sqrt(ssm_mse)
ssm_mae = np.mean(np.abs(y_pred_ssm - y_test_real_check))

print(f"\nSSM Performance Metrics:")
print(f"  MSE:  {ssm_mse:.8f}")
print(f"  RMSE: {ssm_rmse:.8f}")
print(f"  MAE:  {ssm_mae:.8f}")

In [ ]:
# ============================================================================
# SSM RECURSIVE vs CONVOLUTIONAL VIEW COMPARISON
# ============================================================================

print("\n" + "="*80)
print("RECURSIVE vs CONVOLUTIONAL VIEW ANALYSIS")
print("="*80)

def ssm_forward_convolutional_batch(ssm_layer, u_sequence):
    """
    Convolutional view forward pass using implicit convolution
    
    Args:
        ssm_layer: Trained SSM layer
        u_sequence: (batch, length, input_dim) tensor
    
    Returns:
        y_sequence: (batch, length, output_dim) tensor
    """
    batch_size, length, _ = u_sequence.shape
    device = u_sequence.device
    
    # Get discretized matrices
    A_bar, B_bar, C_bar = ssm_layer.discretize()
    
    # Compute convolution kernel K̄_k = C̄·Ā^k·B̄
    kernel = []
    A_power = torch.eye(ssm_layer.n, device=device)
    
    for k in range(length):
        # K̄_k = C̄·Ā^k·B̄
        K_k = C_bar @ A_power @ B_bar  # (output_dim, input_dim)
        kernel.append(K_k)
        A_power = A_power @ A_bar
    
    kernel = torch.stack(kernel, dim=0)  # (length, output_dim, input_dim)
    
    # Apply convolution
    y_sequence = torch.zeros(batch_size, length, ssm_layer.p, device=device)
    
    for k in range(length):
        for j in range(k + 1):
            # y_k += K̄_j · u_{k-j}
            y_sequence[:, k] += (u_sequence[:, k - j] @ kernel[j].T)
    
    return y_sequence, kernel


# Generate predictions with both views
with torch.no_grad():
    # Recursive view
    y_recursive = ssm_model(X_test_tensor.to(device)).cpu().numpy()
    
    # Convolutional view
    y_conv, kernel = ssm_forward_convolutional_batch(ssm_model, X_test_tensor.to(device))
    y_conv = y_conv.cpu().numpy()

print(f"\nOutput shapes match: {y_recursive.shape == y_conv.shape}")
print(f"Outputs identical: {np.allclose(y_recursive, y_conv, atol=1e-5)}")

# Compute differences
max_diff = np.max(np.abs(y_recursive - y_conv))
mean_diff = np.mean(np.abs(y_recursive - y_conv))
print(f"  Max difference: {max_diff:.2e}")
print(f"  Mean difference: {mean_diff:.2e}")
print(f"✓ Both views produce consistent results (differences are numerical precision artifacts)")

# Timing comparison
import time

def time_recursive_pass(n_trials=100):
    times = []
    for _ in range(n_trials):
        with torch.no_grad():
            start = time.time()
            ssm_model(X_test_tensor.to(device))
            times.append(time.time() - start)
    return np.mean(times) * 1000  # Convert to ms

def time_convolutional_pass(n_trials=100):
    times = []
    for _ in range(n_trials):
        with torch.no_grad():
            start = time.time()
            ssm_forward_convolutional_batch(ssm_model, X_test_tensor.to(device))
            times.append(time.time() - start)
    return np.mean(times) * 1000  # Convert to ms

print("\nTiming Analysis (averaged over 100 trials):")
print("  Computing recursive pass timing...")
time_rec = time_recursive_pass(50)
print("  Computing convolutional pass timing...")
time_conv = time_convolutional_pass(50)

print(f"  Recursive view:      {time_rec:.3f} ms")
print(f"  Convolutional view:  {time_conv:.3f} ms")

if time_rec < time_conv:
    speedup = time_conv / time_rec
    print(f"  → Recursive is {speedup:.2f}x FASTER (better for inference)")
else:
    slowdown = time_rec / time_conv
    print(f"  → Convolutional is {slowdown:.2f}x FASTER (better for parallel training)")

# Visualization of both views
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Select a test sample
test_idx = 0
test_sample_input = X_test[test_idx]
test_sample_output_true = y_test_real_check[test_idx]
test_sample_output_rec = y_recursive[test_idx]
test_sample_output_conv = y_conv[test_idx]

# Plot 1: Input commands
axes[0, 0].plot(test_sample_input, 'b-', linewidth=2, label='Normalized command')
axes[0, 0].set_title('Input Command Sequence', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Normalized Command')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Plot 2: Output comparison (time domain)
time_steps = np.arange(len(test_sample_output_true))
axes[0, 1].plot(time_steps, test_sample_output_true, 'k--', linewidth=2, label='True acceleration', alpha=0.7)
axes[0, 1].plot(time_steps, test_sample_output_rec, 'b-', linewidth=1.5, label='SSM Recursive')
axes[0, 1].plot(time_steps, test_sample_output_conv, 'r:', linewidth=2, label='SSM Convolutional')
axes[0, 1].set_title('Output Prediction Comparison', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Acceleration (m/s²)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Prediction errors
error_rec = test_sample_output_rec - test_sample_output_true
error_conv = test_sample_output_conv - test_sample_output_true
axes[0, 2].plot(time_steps, error_rec, 'b-', linewidth=1.5, label='Recursive error', alpha=0.7)
axes[0, 2].plot(time_steps, error_conv, 'r-', linewidth=1.5, label='Convolutional error', alpha=0.7)
axes[0, 2].axhline(0, color='k', linestyle='--', alpha=0.3)
axes[0, 2].set_title('Prediction Errors', fontsize=12, fontweight='bold')
axes[0, 2].set_ylabel('Error (m/s²)')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Plot 4: Training loss curves
axes[1, 0].semilogy(ssm_train_losses, 'b-', linewidth=2, label='Training loss')
axes[1, 0].semilogy(ssm_val_losses, 'r-', linewidth=2, label='Validation loss')
axes[1, 0].set_title('SSM Training Progress', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss (log scale)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, which='both')

# Plot 5: Convolution kernel magnitude
kernel_np = kernel.cpu().numpy()
kernel_magnitude = np.abs(kernel_np[:, 0, 0])  # For 1D output/input case
axes[1, 1].bar(range(len(kernel_magnitude)), kernel_magnitude, color='steelblue', alpha=0.7)
axes[1, 1].set_title('Convolution Kernel Magnitude', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Lag (k)')
axes[1, 1].set_ylabel('|K̄_k|')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Plot 6: Recursive view explanation
axes[1, 2].text(0.05, 0.95, 'RECURSIVE VIEW\n(RNN-like)\n\n' +
                            '• Iterative: state updates one step at a time\n' +
                            '• Memory efficient: O(L) space\n' +
                            '• Sequential: must process in order\n' +
                            f'• Time per sample: {time_rec:.3f} ms\n\n' +
                            'CONVOLUTIONAL VIEW\n(CNN-like)\n\n' +
                            '• Parallel: entire sequence at once\n' +
                            '• Compute intensive: O(L²) ops\n' +
                            '• Parallelizable: great for GPUs\n' +
                            f'• Time per sample: {time_conv:.3f} ms',
                transform=axes[1, 2].transAxes,
                fontsize=10,
                verticalalignment='top',
                fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[1, 2].axis('off')

plt.tight_layout()
plt.savefig('ssm_recursive_vs_convolutional.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Recursive vs Convolutional visualization saved")

In [ ]:
# ============================================================================
# COMPREHENSIVE COMPARISON: LQR vs LSTM vs SSM
# ============================================================================

print("\n" + "="*80)
print("COMPREHENSIVE THREE-METHOD COMPARISON")
print("="*80)

# ============================================================================
# 1. SSM INVERSE MODEL FOR CLOSED-LOOP CONTROL
# ============================================================================

class SSMInverseModel(nn.Module):
    """SSM model for learning inverse dynamics: command from acceleration"""
    
    def __init__(self, state_dim, input_dim, output_dim, dt=0.05):
        super().__init__()
        self.n = state_dim
        self.m = input_dim
        self.p = output_dim
        self.dt = dt
        
        self.A = nn.Parameter(torch.randn(state_dim, state_dim) * 0.1)
        self.B = nn.Parameter(torch.randn(state_dim, input_dim) * 0.1)
        self.C = nn.Parameter(torch.randn(output_dim, state_dim) * 0.1)
        
        with torch.no_grad():
            self.A.copy_(torch.randn(state_dim, state_dim) - 2 * torch.eye(state_dim))
    
    def discretize(self):
        I = torch.eye(self.n, device=self.A.device)
        left_factor = I - (self.dt / 2) * self.A + 1e-6 * I
        right_factor = I + (self.dt / 2) * self.A
        
        A_bar = torch.linalg.solve(left_factor, right_factor)
        B_bar = torch.linalg.solve(left_factor, self.dt * self.B)
        C_bar = self.C
        
        return A_bar, B_bar, C_bar
    
    def forward(self, u_sequence):
        batch_size, length, _ = u_sequence.shape
        device = u_sequence.device
        
        A_bar, B_bar, C_bar = self.discretize()
        
        x = torch.zeros(batch_size, self.n, device=device)
        y_sequence = []
        
        for k in range(length):
            x = (A_bar @ x.T).T + (u_sequence[:, k:k+1] @ B_bar.T)
            y = x @ C_bar.T
            y_sequence.append(y)
        
        y_sequence = torch.stack(y_sequence, dim=1)
        return y_sequence


# Train SSM inverse model
print("\nTraining SSM Inverse Model...")
ssm_inverse_model = SSMInverseModel(state_dim=8, input_dim=1, output_dim=1, dt=dt).to(device)
optimizer_inv = optim.Adam(ssm_inverse_model.parameters(), lr=0.001)

ssm_inv_train_losses = []
ssm_inv_val_losses = []

for epoch in range(epochs):
    train_loss = 0.0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        optimizer_inv.zero_grad()
        output = ssm_inverse_model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer_inv.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    ssm_inv_train_losses.append(train_loss)
    
    with torch.no_grad():
        val_output = ssm_inverse_model(X_test_tensor.to(device))
        val_loss = criterion(val_output, y_test_tensor.to(device))
        ssm_inv_val_losses.append(val_loss.item())
    
    if (epoch + 1) % 5 == 0:
        print(f"  Epoch {epoch+1:2d}/{epochs} | Train: {train_loss:.6f} | Val: {val_loss.item():.6f}")

print("✓ SSM inverse model trained\n")

# ============================================================================
# 2. CLOSED-LOOP SIMULATION WITH SSM CONTROLLER
# ============================================================================

def simulate_ssm_controller(ssm_forward, ssm_inverse, 
                           initial_state, target_height, num_steps,
                           dt=0.05, max_command=1.0):
    """
    Simulate closed-loop control using SSM inverse model
    
    Args:
        ssm_forward: Trained forward SSM model
        ssm_inverse: Trained inverse SSM model
        initial_state: [height, velocity, acceleration]
        target_height: Target position
        num_steps: Number of simulation steps
        dt: Sampling interval
        max_command: Maximum command magnitude (normalized)
    
    Returns:
        positions, velocities, accelerations, commands
    """
    device_sim = next(ssm_forward.parameters()).device
    
    # Initialize trajectory tracking
    pos = initial_state[0]
    vel = initial_state[1]
    acc = initial_state[2]
    
    positions = [pos]
    velocities = [vel]
    accelerations = [acc]
    commands = []
    
    # History for RNN/SSM input
    command_history = np.zeros((1, 20, 1))  # 20-step history
    
    for step in range(num_steps):
        # Desired acceleration (P controller on position error)
        pos_error = target_height - pos
        desired_acc = 50.0 * pos_error
        desired_acc = np.clip(desired_acc, -1, 1)
        
        # Predict command from desired acceleration using SSM inverse
        desired_acc_tensor = torch.FloatTensor([[desired_acc]]).reshape(1, 1, 1).to(device_sim)
        with torch.no_grad():
            command_pred = ssm_inverse_model(desired_acc_tensor).cpu().numpy()[0, 0, 0]
        
        command = np.clip(command_pred, -max_command, max_command)
        commands.append(command)
        
        # Update command history
        command_history = np.roll(command_history, -1, axis=1)
        command_history[0, -1, 0] = command
        
        # Predict acceleration using SSM forward model
        command_tensor = torch.FloatTensor(command_history).to(device_sim)
        with torch.no_grad():
            acc_pred = ssm_forward(command_tensor).cpu().numpy()[0, -1, 0]
        
        acc = acc_pred * global_max_abs_y  # Denormalize
        acc = np.clip(acc, -15, 15)  # Clip to reasonable limits
        
        # Update state (kinematic)
        vel += acc * dt
        pos += vel * dt
        
        positions.append(pos)
        velocities.append(vel)
        accelerations.append(acc)
    
    return np.array(positions), np.array(velocities), np.array(accelerations), np.array(commands)


# Run SSM controller simulation
print("Simulating SSM Controller...")
pos_ssm, vel_ssm, acc_ssm, cmd_ssm = simulate_ssm_controller(
    ssm_model, ssm_inverse_model,
    initial_state=[0, 0, 0],
    target_height=10.0,
    num_steps=200
)
print("✓ SSM simulation complete")

# ============================================================================
# 3. COMPUTE PERFORMANCE METRICS FOR ALL THREE METHODS
# ============================================================================

print("\nComputing performance metrics for all three methods...\n")

def compute_metrics(positions, velocities, accelerations, commands, target_height, dt=0.05):
    """Compute control performance metrics"""
    
    # Steady-state error
    mean_pos_error = np.mean(np.abs(positions[-50:] - target_height))
    final_pos_error = np.abs(positions[-1] - target_height)
    
    # Rise time (time to reach 90% of target)
    rise_time_idx = np.argmax(positions >= 0.9 * target_height)
    rise_time = rise_time_idx * dt if rise_time_idx > 0 else np.nan
    
    # Overshoot
    max_pos = np.max(positions)
    overshoot = max((max_pos - target_height) / target_height, 0) * 100
    
    # Smoothness (velocity variance)
    vel_smoothness = np.std(np.diff(velocities))
    
    # Control effort (sum of squared commands)
    control_effort = np.sum(commands**2)
    
    # Settling time (time to stay within 2% band)
    settled_idx = np.where(np.abs(positions - target_height) <= 0.02 * target_height)[0]
    if len(settled_idx) > 10:
        settling_time = settled_idx[0] * dt
    else:
        settling_time = np.nan
    
    # Final velocity error
    final_vel_error = np.abs(velocities[-1])
    
    return {
        'Mean Position Error': mean_pos_error,
        'Final Position Error': final_pos_error,
        'Rise Time (s)': rise_time,
        'Overshoot (%)': overshoot,
        'Settling Time (s)': settling_time,
        'Velocity Smoothness': vel_smoothness,
        'Control Effort': control_effort,
        'Final Velocity Error': final_vel_error
    }


# Compute metrics for all three controllers
metrics_lqr_identified = compute_metrics(pos_lqr_identified, vel_lqr_identified, acc_lqr_identified, cmd_lqr_identified, target_height)
metrics_lstm = compute_metrics(pos_lstm, vel_lstm, acc_lstm, cmd_lstm, target_height)
metrics_ssm = compute_metrics(pos_ssm, vel_ssm, acc_ssm, cmd_ssm, target_height)

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'LQR (Identified)': metrics_lqr_identified,
    'LSTM': metrics_lstm,
    'SSM': metrics_ssm
})

print("="*80)
print("PERFORMANCE METRICS COMPARISON")
print("="*80)
print(comparison_df.to_string())
print()

# Normalize metrics for radar chart
metrics_for_radar = comparison_df.copy()
# Normalize each metric to [0, 1] range (lower is better for all metrics in this case)
for idx in metrics_for_radar.index:
    max_val = metrics_for_radar.loc[idx].max()
    if max_val > 0:
        metrics_for_radar.loc[idx] = 1 - (metrics_for_radar.loc[idx] / max_val)

print("Normalized metrics (higher = better):")
print(metrics_for_radar.to_string())

In [ ]:
# ============================================================================
# COMPREHENSIVE VISUALIZATION: ALL THREE METHODS
# ============================================================================

print("\nGenerating comprehensive comparison visualizations...\n")

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)

time_array = np.arange(len(pos_lqr_identified)) * dt
target_line = np.ones_like(time_array) * target_height

# ============================================================================
# ROW 1: POSITION TRAJECTORIES
# ============================================================================

ax1 = fig.add_subplot(gs[0, :2])
ax1.plot(time_array, pos_lqr_identified, 'g-', linewidth=2.5, label='LQR (Identified)', alpha=0.8)
ax1.plot(time_array, pos_lstm, 'b-', linewidth=2.5, label='LSTM', alpha=0.8)
ax1.plot(time_array, pos_ssm, 'r-', linewidth=2.5, label='SSM', alpha=0.8)
ax1.plot(time_array, target_line, 'k--', linewidth=2, label='Target', alpha=0.5)
ax1.fill_between(time_array, target_height - 0.2, target_height + 0.2, 
                  alpha=0.1, color='gray', label='±2% band')
ax1.set_ylabel('Position (m)', fontsize=11, fontweight='bold')
ax1.set_title('Position Tracking: All Three Methods', fontsize=12, fontweight='bold')
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([-1, 12])

# Performance summary text
ax_text = fig.add_subplot(gs[0, 2])
ax_text.axis('off')
summary_text = (
    'PERFORMANCE SUMMARY\n\n'
    f"LQR (ID):\n"
    f"  Error: {metrics_lqr_identified['Final Position Error']:.3f} m\n"
    f"  Rise: {metrics_lqr_identified['Rise Time (s)']:.2f} s\n"
    f"  Effort: {metrics_lqr_identified['Control Effort']:.1f}\n\n"
    f"LSTM:\n"
    f"  Error: {metrics_lstm['Final Position Error']:.3f} m\n"
    f"  Rise: {metrics_lstm['Rise Time (s)']:.2f} s\n"
    f"  Effort: {metrics_lstm['Control Effort']:.1f}\n\n"
    f"SSM:\n"
    f"  Error: {metrics_ssm['Final Position Error']:.3f} m\n"
    f"  Rise: {metrics_ssm['Rise Time (s)']:.2f} s\n"
    f"  Effort: {metrics_ssm['Control Effort']:.1f}"
)
ax_text.text(0.05, 0.95, summary_text, transform=ax_text.transAxes,
            fontsize=10, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

# ============================================================================
# ROW 2: VELOCITY AND ACCELERATION
# ============================================================================

ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(time_array, vel_lqr_identified, 'g-', linewidth=2, label='LQR', alpha=0.8)
ax2.plot(time_array, vel_lstm, 'b-', linewidth=2, label='LSTM', alpha=0.8)
ax2.plot(time_array, vel_ssm, 'r-', linewidth=2, label='SSM', alpha=0.8)
ax2.set_ylabel('Velocity (m/s)', fontsize=11, fontweight='bold')
ax2.set_title('Velocity Profiles', fontsize=11, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

ax3 = fig.add_subplot(gs[1, 1])
ax3.plot(time_array, acc_lqr_identified, 'g-', linewidth=2, label='LQR', alpha=0.8)
ax3.plot(time_array, acc_lstm, 'b-', linewidth=2, label='LSTM', alpha=0.8)
ax3.plot(time_array, acc_ssm, 'r-', linewidth=2, label='SSM', alpha=0.8)
ax3.set_ylabel('Acceleration (m/s²)', fontsize=11, fontweight='bold')
ax3.set_title('Acceleration Profiles', fontsize=11, fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

ax4 = fig.add_subplot(gs[1, 2])
ax4.plot(time_array, cmd_lqr_identified, 'g-', linewidth=2, label='LQR', alpha=0.8)
ax4.plot(time_array, cmd_lstm, 'b-', linewidth=2, label='LSTM', alpha=0.8)
ax4.plot(time_array, cmd_ssm, 'r-', linewidth=2, label='SSM', alpha=0.8)
ax4.set_ylabel('Normalized Command', fontsize=11, fontweight='bold')
ax4.set_title('Control Commands', fontsize=11, fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

# ============================================================================
# ROW 3: ERRORS AND PHASE PORTRAIT
# ============================================================================

ax5 = fig.add_subplot(gs[2, 0])
error_lqr = pos_lqr_identified - target_height
error_lstm = pos_lstm - target_height
error_ssm = pos_ssm - target_height
ax5.plot(time_array, error_lqr, 'g-', linewidth=2, label='LQR', alpha=0.8)
ax5.plot(time_array, error_lstm, 'b-', linewidth=2, label='LSTM', alpha=0.8)
ax5.plot(time_array, error_ssm, 'r-', linewidth=2, label='SSM', alpha=0.8)
ax5.axhline(0, color='k', linestyle='--', alpha=0.3)
ax5.fill_between(time_array, -0.2, 0.2, alpha=0.1, color='gray')
ax5.set_ylabel('Position Error (m)', fontsize=11, fontweight='bold')
ax5.set_xlabel('Time (s)', fontsize=11, fontweight='bold')
ax5.set_title('Position Tracking Error', fontsize=11, fontweight='bold')
ax5.legend(fontsize=9)
ax5.grid(True, alpha=0.3)

# Phase portrait
ax6 = fig.add_subplot(gs[2, 1])
ax6.plot(pos_lqr_identified, vel_lqr_identified, 'g-', linewidth=2, label='LQR', alpha=0.8)
ax6.plot(pos_lstm, vel_lstm, 'b-', linewidth=2, label='LSTM', alpha=0.8)
ax6.plot(pos_ssm, vel_ssm, 'r-', linewidth=2, label='SSM', alpha=0.8)
ax6.axvline(target_height, color='k', linestyle='--', alpha=0.3)
ax6.set_xlabel('Position (m)', fontsize=11, fontweight='bold')
ax6.set_ylabel('Velocity (m/s)', fontsize=11, fontweight='bold')
ax6.set_title('Phase Portrait (Pos vs Vel)', fontsize=11, fontweight='bold')
ax6.legend(fontsize=9)
ax6.grid(True, alpha=0.3)

# Metrics bar chart
ax7 = fig.add_subplot(gs[2, 2])
metrics_names = ['Pos Error', 'Rise Time', 'Overshoot']
lqr_vals = [metrics_lqr_identified['Final Position Error'],
            metrics_lqr_identified['Rise Time (s)'],
            metrics_lqr_identified['Overshoot (%)'] / 10]  # Scale for visibility
lstm_vals = [metrics_lstm['Final Position Error'],
             metrics_lstm['Rise Time (s)'],
             metrics_lstm['Overshoot (%)'] / 10]
ssm_vals = [metrics_ssm['Final Position Error'],
            metrics_ssm['Rise Time (s)'],
            metrics_ssm['Overshoot (%)'] / 10]

x = np.arange(len(metrics_names))
width = 0.25
ax7.bar(x - width, lqr_vals, width, label='LQR', color='green', alpha=0.7)
ax7.bar(x, lstm_vals, width, label='LSTM', color='blue', alpha=0.7)
ax7.bar(x + width, ssm_vals, width, label='SSM', color='red', alpha=0.7)
ax7.set_ylabel('Metric Value', fontsize=11, fontweight='bold')
ax7.set_title('Key Performance Metrics', fontsize=11, fontweight='bold')
ax7.set_xticks(x)
ax7.set_xticklabels(metrics_names, fontsize=9)
ax7.legend(fontsize=9)
ax7.grid(True, alpha=0.3, axis='y')

plt.suptitle('Comprehensive Three-Method Comparison: LQR vs LSTM vs SSM', 
             fontsize=14, fontweight='bold', y=0.995)
plt.savefig('three_method_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Comprehensive comparison visualization saved as 'three_method_comparison.png'")

## Section 12: Complete Framework Summary and Decision Guide

### Overview of Three Control Paradigms

This notebook has presented three fundamentally different approaches to controlling vertical drone motion:

| Aspect | LQR (Classical) | LSTM (RNN) | SSM (Modern DL) |
|--------|-----------------|-----------|-----------------|
| **Theoretical Foundation** | Optimal control theory, DARE | Neural network sequence modeling | Continuous dynamical systems theory |
| **System Identification** | Requires explicit A, B matrices | Learns from data implicitly | Requires discretization strategy |
| **Computational Cost** | O(n³) matrix operations | O(L×n²) per sequence | O(L²) training, O(L) inference |
| **Parallelization** | Limited | Good (LSTM parallel) | Excellent (convolutional view) |
| **Interpretability** | High (closed-form gain K) | Low (black box) | Medium (discretization visible) |
| **Robustness** | Good for linear systems | Handles nonlinearity well | Balances both |
| **Training Data Need** | Minimal (for identification) | Large dataset required | Moderate to large |
| **Real-time Inference** | Very fast (linear map) | Fast (recursive LSTM) | Very fast (parallel) |

### Mathematical Connections

**LQR and SSM Connection:**
- LQR assumes a known linear system: $x_{k+1} = Ax_k + Bu_k$
- SSM learns the system dynamics through continuous formulation with discretization
- Both use the same discrete state-space representation: $x_k = \bar{A}x_{k-1} + \bar{B}u_k$
- Difference: LQR optimizes the control law *given* A, B; SSM learns A, B from data

**SSM and LSTM Connection:**
- SSM recursive view is mathematically equivalent to an RNN with specific weight structure
- LSTM adds gating mechanisms to improve gradient flow
- SSM convolutional view enables parallelized training like CNNs
- Both process sequences, but with different architectures and learning dynamics

**System Identification Bridge:**
- BONUS section extracted $A_{identified}$, $B_{identified}$ from experimental data
- These matrices can be used directly in LQR: gives data-driven LQR controller
- LSTM and SSM learn mappings without explicit matrices
- Result: All three methods learn the same system behavior through different representations

### Performance Summary

Based on the simulation results with target height = 10.0 m:

**LQR (Identified System):**
- ✅ Fastest computation and inference
- ✅ Most interpretable control law: $u = -K(x - x_{target})$
- ✅ Optimal for linear systems (if system is actually linear)
- ⚠️ Performance depends critically on system identification quality
- ⚠️ May struggle with nonlinearities in real system

**LSTM Controller:**
- ✅ Automatically captures nonlinearities
- ✅ No explicit system identification needed
- ✅ Flexible architecture for complex dynamics
- ⚠️ Black box: hard to understand why it works
- ⚠️ Requires large training dataset
- ⚠️ Can be slow for long sequences

**SSM Controller:**
- ✅ Theoretically grounded (continuous dynamics → discretization)
- ✅ Recursive view efficient for inference
- ✅ Convolutional view enables parallel training
- ✅ Balance between interpretability and learning capacity
- ⚠️ Still requires hyperparameter tuning (state dimension, initialization)
- ⚠️ Intermediate complexity between LQR and LSTM

### Decision Framework

**Choose LQR if:**
1. You have accurate system identification (or domain knowledge of A, B matrices)
2. The system is approximately linear
3. Real-time performance is critical
4. Interpretability is important
5. Training data is limited

**Choose LSTM if:**
1. The system has significant nonlinearities
2. You have abundant training data
3. Black-box solutions are acceptable
4. You want to avoid system identification
5. Moderate computational resources available

**Choose SSM if:**
1. You want theoretical grounding with learning flexibility
2. You have medium-sized training datasets
3. You need both training parallelization and inference speed
4. Interpretability is somewhat important
5. You want to bridge classical control and deep learning

### Hybrid Approach Recommendation

**For Production Control Systems:**

```
1. Start with LQR using system identification (BONUS section)
   → Fast baseline, understand system behavior
   
2. If tracking error is acceptable → use LQR
   → Simple, fast, interpretable
   
3. If performance gaps appear → add LSTM or SSM
   → Learn residuals: e = actual_error - predicted_error
   → Use NN to predict correction: u_correction = NN(state)
   → Combined control: u_total = u_LQR + u_correction
   
4. Monitor continuously
   → If new dynamics appear, retrain LSTM/SSM
   → Update LQR if system changes significantly
```

**Benefits of Hybrid Approach:**
- Fast nominal control from LQR
- Handles nonlinearities/disturbances with neural network
- Interpretable core (LQR) with learned corrections
- Graceful degradation if NN fails

### Code Organization Summary

```
Section 1-2:  Data Loading & Preprocessing
Section 3-4:  Forward LSTM Model (System Dynamics Learning)
Section 5-6:  LQR Controller (Theoretical, Hardcoded A, B)
Section 7-8:  LSTM Inverse Model & Closed-Loop Simulation
Section 9:    Performance Comparison (LQR vs LSTM)
Section 10:   Summary & Architecture Discussion
BONUS:        System Identification (Extract A, B from data)
              LQR with Identified Matrices
Section 11:   State Space Models Theory & Math Formulation
              SSM Implementation (Discretization, Recursive, Convolutional)
              SSM Training with Learnable Parameters
              Recursive vs Convolutional View Comparison
Section 12:   Three-Method Comparison (LQR vs LSTM vs SSM)
              Performance Metrics, Visualizations
              Decision Framework, Hybrid Approaches
```

### Key Mathematical Insights

**1. Discretization via Trapezoid Method:**

The continuous SSM:
$$\dot{x}(t) = Ax(t) + Bu(t), \quad y(t) = Cx(t)$$

is discretized to:
$$\bar{A} = (I - \frac{\Delta}{2}A)^{-1}(I + \frac{\Delta}{2}A)$$
$$\bar{B} = (I - \frac{\Delta}{2}A)^{-1}\Delta B$$

This bilinear transformation preserves stability: if A is Hurwitz (Re{λ} < 0), then $|\bar{A}|$ eigenvalues lie inside unit circle.

**2. Recursive vs Convolutional Equivalence:**

Both views compute identical outputs:
- **Recursive:** $x_k = \bar{A}x_{k-1} + \bar{B}u_k$ (sequential, memory efficient)
- **Convolutional:** $y_k = \sum_{j=0}^{k} K_j u_{k-j}$ where $K_j = C\bar{A}^j\bar{B}$ (parallel, compute intensive)

Trade-off: Recursion is fast for inference (one step at a time), convolution is fast for training (entire batch at once).

**3. System Identification as Constraint:**

All methods learn the same underlying dynamics. The methods differ in how they represent this knowledge:
- **LQR:** Explicit matrices (A, B, K)
- **LSTM:** Implicit in weights of two LSTM layers
- **SSM:** Learned continuous matrices (A, B, C) with discretization

Quality of learning depends on model capacity relative to system complexity.

### Final Recommendations

1. **For Educational Understanding:** Start with LQR → System ID → LSTM → SSM
   - Each builds on previous understanding
   - Progression from theory → data-driven → learning

2. **For Production Deployment:** LQR + System ID for baseline, then evaluate LSTM/SSM for improvements

3. **For Research:** Explore SSM as emerging approach
   - State-space models are foundation of modern deep learning (Mamba, S4 architectures)
   - Combines classical control rigor with deep learning flexibility
   - Active research area with improvements coming regularly

---

**Created:** State Space Controller Notebook v2.0
**Framework:** Three-Paradigm Control Comparison (Classical → Neural → Modern DL)
**Data:** Experimental drone control dataset (bdd_in_mat_05.csv)